**Ogólne założenia projektu:**
* eksploracja danych (jeśli są nowe), można początkowo (ale nie trzeba) skorzystać z sql’a, ale potem już w pythonie
* wybór interesujących cech
* zastosowanie podstawowych metod statystycznych
* wprowadzenie analizy czynnikowej (PCA)
* wykonanie wykresów, wizualizacji w pythonie
* zastosowanie magic/widget
* zastosowanie obiektowości w aplikacji
Mistrzowski zestaw pokemonów
https://www.kaggle.com/lrcusack/pokemontrainers#database.sqlite


**Cel Statystyczny:** Ograniczając się do pewnej, maksymalnej liczby zebranych pokemonów, należy określić potencjalnie najlepszy zestaw w starciu z jakimkolwiek innym przeciwnikiem.


**Cel Python:** Symulator walki

In [ ]:
# wczytanie danych
import pandas as pd

pokemony = pd.read_csv('main_Pokemon.csv')
trenerzy = pd.read_csv('main_Trainers.csv')

In [ ]:
# wyświetlenie pierwszych 10 rekordów
pokemony.head(10)

In [ ]:
# podstawowe informacje o bazie pokemonów
pokemony.info()

In [ ]:
# wyznaczenie liczby możliwych kombinacji zestawu 6 pokemonów
from math import factorial

n =len(pokemony['pokename'].unique())
print(n)
k = 6
liczba_kombinacji = factorial(n)/(factorial(k)*factorial(n-k))
print('{:.2E}'.format(liczba_kombinacji))

In [ ]:
# udziały procentowe wg typu
pokemony['type1'].value_counts() # zlicza i jednocześnie grupuje rekordy w kolumnie
udzialy_procentowe = round(pokemony['type1'].value_counts(normalize=True)*100,2)
pierwsze = 8
print(udzialy_procentowe[:pierwsze])
print()
udzial_procentowy_wybranych = udzialy_procentowe[:pierwsze].sum()
print('Wybrane typy stanowią {}% populacji '.format(udzial_procentowy_wybranych))

In [ ]:
from matplotlib import pyplot as plt
# from numpy import arange
plt.rcParams.update({'font.size': 15})

liczebnosc = pokemony['type1'].value_counts()
values = liczebnosc.values
labels = liczebnosc.index.values

# fig, ax = plt.subplots()
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)

for label, value in zip(labels, values):
    ax.bar(label, value, 0.75)

ax.set_xticklabels(labels, rotation=90)
# ax.set_xticklabels(labels, rotation=90, fontsize=20)
# ax.set_yticklabels(fontsize=20)
# matplotlib.rc('xtick', labelsize=20) 
# matplotlib.rc('ytick', labelsize=20)

plt.title('Liczebność pokemonów wg typu', fontsize=30)
plt.show()

In [ ]:
# udziały procentowe wg typu dla 'pokelevel' = 100
level_100 = pokemony[pokemony['pokelevel'] == 100]
level_100_proc = round(level_100['type1'].value_counts(normalize=True)*100,2)
level_100_proc

In [ ]:
from matplotlib import pyplot as plt
# from numpy import arange
plt.rcParams.update({'font.size': 15})

liczebnosc_100 = pokemony[pokemony['pokelevel'] == 100]['type1'].value_counts()
values = liczebnosc_100.values
labels = liczebnosc_100.index.values

# fig, ax = plt.subplots()
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)

for label, value in zip(labels, values):
    ax.bar(label, value, 0.75)

ax.set_xticklabels(labels, rotation=90)
# ax.set_xticklabels(labels, rotation=90, fontsize=20)
# ax.set_yticklabels(fontsize=20)
# matplotlib.rc('xtick', labelsize=20) 
# matplotlib.rc('ytick', labelsize=20)

plt.title('Liczebność pokemonów wg typu dla pokelevel = 100', fontsize=30)
plt.show()

In [ ]:
# udziały procentowe wg gatunku
round(pokemony['pokename'].value_counts(normalize=True)[:20]*100,2)

# wniosek: analizowanie pokemonów wg gatunku nie ma sensu (zbyt równomierny rozkład)

In [ ]:
from matplotlib import pyplot as plt
# from numpy import arange
plt.rcParams.update({'font.size': 15})

liczebnosc_gatunki = pokemony['pokename'].value_counts()[:20]
values = liczebnosc_gatunki.values
labels = liczebnosc_gatunki.index.values

# fig, ax = plt.subplots()
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(1,1,1)

for label, value in zip(labels, values):
    ax.bar(label, value, 0.75)

ax.set_xticklabels(labels, rotation=90)
# ax.set_xticklabels(labels, rotation=90, fontsize=20)
# ax.set_yticklabels(fontsize=20)
# matplotlib.rc('xtick', labelsize=20) 
# matplotlib.rc('ytick', labelsize=20)

plt.title('Liczebność pokemonów wg gatunku', fontsize=30)
plt.show()

In [ ]:
# max(pokelevel) w ramach podziału na gatunki
unikalne_top = pokemony.loc[pokemony.groupby('pokename')['pokelevel'].idxmax()]
unikalne_top

In [ ]:
# max(pokelevel) w ramach podziału na gatunki
# max(attack) w ramach podziału na typ
best_attack = unikalne_top.loc[unikalne_top.groupby('type1')['attack'].idxmax()]
best_attack

In [ ]:
# wczytanie tabeli współczynników ataku
attack_table = pd.read_csv('attack_table.csv')
attack_table

In [ ]:
# akumulator wspomagający wybór drużyny ofensywnej
wszystkie_typy = pokemony['type1'].unique()
istotne_typy = udzialy_procentowe.index.values[:8]

ranking_atak = {}
for typ in wszystkie_typy:
    rate = 0
    for przeciwnik in istotne_typy:
        zestaw = attack_table[(attack_table['attack']==typ) & (attack_table['defense']==przeciwnik)]
        if zestaw['effectiveness'].values==2:
            rate+=1
    ranking_atak[typ]=rate
ranking_atak           

In [ ]:
# wybrane typy pokemenów drużyny ofensywnej
# skład drużyny ofensywnej
attack_types = ['Fighting', 'Fire', 'Flying', 'Grass', 'Ground', 'Rock']

attack_team = best_attack[best_attack['type1'].isin(attack_types)]
attack_team

In [ ]:
# max(pokelevel) w ramach podziału na gatunki
# max(defense) w ramach podziału na typ
best_defense = unikalne_top.loc[unikalne_top.groupby('type1')['defense'].idxmax()]
best_defense

In [ ]:
# akumulator wspomagający wybór drużyny defensywnej
wszystkie_typy = pokemony['type1'].unique()
istotne_typy = udzialy_procentowe.index.values[:8]

ranking_obrona = {}
for typ in wszystkie_typy:
    rate = 0
    for przeciwnik in istotne_typy:
        zestaw = attack_table[(attack_table['attack']==przeciwnik) & (attack_table['defense']==typ)]
        if zestaw['effectiveness'].values==0.5:
            rate+=1
    ranking_obrona[typ]=rate
ranking_obrona    

In [ ]:
# wybrane typy pokemenów drużyny defensywnej
# skład drużyny defensywnej
defense_types = ['Dragon', 'Fire', 'Grass', 'Poison', 'Rock', 'Steel']

defense_team = best_defense[best_defense['type1'].isin(defense_types)]
defense_team

In [ ]:
# zgrubna analiza dla 5 przedziałów 'pokelevel'
level_2_20 = pokemony[(pokemony['pokelevel'] <= 20)]
level_21_40 = pokemony[(pokemony['pokelevel'] > 20) & (pokemony['pokelevel'] <= 40)]
level_41_60 = pokemony[(pokemony['pokelevel'] > 40) & (pokemony['pokelevel'] <= 60)]
level_61_80 = pokemony[(pokemony['pokelevel'] > 60) & (pokemony['pokelevel'] <= 80)]
level_81_100 = pokemony[(pokemony['pokelevel'] > 80)]

przedziały = (level_2_20, level_21_40, level_41_60, level_61_80, level_81_100)

for i in przedziały:
    print('pokelevel:', i['pokelevel'].min(), '-', i['pokelevel'].max())
    print('hp:', i['hp'].min(), '-', i['hp'].max())
    print('attack:', i['attack'].min(), '-', i['attack'].max())
    print('defense:', i['defense'].min(), '-', i['defense'].max())
    print('liczba_pokemonów:', i['trainer_id'].count(), '\n')


In [ ]:
# from matplotlib import pyplot as plt

# %matplotlib inline

# pokemony['type1'].value_counts().plot.bar()

# plt.bar(labels, values)

# for label, value in zip(labels, values):
#     plt.bar(label, value)

# plt.show()